# SAS Interview presentation - Daniele Retico
---
## Building user defined function to generate loans cash flow
---
### Amortizing and bullet loans cash flow generation
---

The main objective of this few lines of code is to present a prototype of function that produces three different kind of loan:

+ plain amortizing loans french style
+ bullet loans
+ deferred amortizing loans french style

**loan_amortizationVectorizedV2** is a function that returns an amortizing plan based on several arguments:

+ **S**: amount borrowed
+ **i**: annual interest rate
+ **T**: loan maturity expressed in years fraction
+ **freq**: payments frequency
+ **preAmm**: identifies the numbers of periods since the notional payment are deferred starting from $t = 0$.


## Mathematical relationships used in loan_amortizationVectorizedV2
---
### French amortization style basics ingredients

1. $S = \sum \limits _{j=1} ^{m} R (1 + i)^{j} $

2. $a_|mi = \frac{1 - (1 + i)^{-m}}{i} $


3. $R = \frac{S}{\sum \limits _{j=1} ^{m}(1 + i)^{j}} $


## Coding part

In [21]:
# loading packages
import numpy as np
import pandas as pd

In [22]:
def loan_amortizationVectorizedV2(S, i, T, freq, preAmm):
    """


    Parameters
    ----------
    S : float
        amount borrowed.
    i : float
        Annual interest rate.
    T : float
        Maturity.
    Freq : float
        Notional payment frequency: 1: Annual
                                    12 Monthly
                                    4  Quaterly
                                    6  Semiannual
    preAmm : Int
            number of periods since amortization occurs starting from t = 0
            
    Raises
    ------
    ValueError
        Inputs value must be coherent with params indications.

    Returns
    -------
    df_loan : DataFrame
        returns a DataFrame object.

    """


    if (S < 0) | (i < 0) | (T < 0) | (freq <= 0) | (preAmm < 0):  # input params control
        raise ValueError("Inputs value must be coherent with params indications")
    else:
        if preAmm == 0:  # Case: Amortization not delayed
            im = (1 + i)**(1/freq) - 1   # interest rate compounding equivalency
            a_freqmi = (1 - (1 + i)**(-T))/im  # annuity calculation
            R = S/a_freqmi  # amortizing amount payid every period

            # initializing times vector

            k = -np.arange(freq * T + 1, 0, -1) 
            k[0] = 0  # debt is repaid at the end of the period, a t = 0 no notional payments

            # initialization and calculation of discount factors
            dcf_kP = np.zeros((len(k), ), dtype=np.float64) + 1 + im
            dcf_kP = np.power(dcf_kP, k.T)
            dcf_kP[0] = 0

            # initializing and calculating notional and interest payment, debt after notional payment respectively (C_k, I_k, D_k)
            # vectors initialization
            C_k = np.zeros((len(k), ), dtype=np.float64)
            I_k = np.zeros((len(k), ), dtype=np.float64)
            D_k = np.zeros((len(k), ), dtype=np.float64)
            C_k = R * dcf_kP  # notional payments vector

            I_k = R * (1 - dcf_kP)  # interest payments vector
            I_k[0] = 0  # no interest payment t = 0

            D_k = np.repeat(S, len(k)) - (np.cumsum(C_k))  # debt amount before notional payments vector

            # initialazing DataFrame object to store results

            dfC_k = pd.DataFrame(C_k)
            dfI_k = pd.DataFrame(I_k)
            dfD_k = pd.DataFrame(D_k)
            df_loan = pd.DataFrame()
            df_loan['Notional'] = dfC_k
            df_loan['Interest'] = dfI_k
            df_loan['Payment notional/intererest'] = dfC_k + dfI_k
            df_loan['Debt amount'] = dfD_k
            return df_loan

        else:
            k = -np.arange(freq * T + 1, 0, -1)
            if len(k) - preAmm == 1:  # bullet loan condition
                R_k = np.zeros((len(k), ), dtype=np.float64)
                R_k[1:len(k)] = S*i
                I_k = R_k
                C_k = np.zeros((len(k), ), dtype=np.float64)
                C_k[len(k) - 1] = S
                D_k = np.repeat(S, len(k))
                D_k[len(k) - 1] = 0

                dfC_k = pd.DataFrame(C_k)
                dfI_k = pd.DataFrame(I_k)
                dfD_k = pd.DataFrame(D_k)
                df_loan = pd.DataFrame()
                df_loan['Notional'] = dfC_k
                df_loan['Interest'] = dfI_k
                df_loan['Payment notional/intererest'] = dfC_k + dfI_k
                df_loan['Debt amount'] = dfD_k
                return df_loan
            else:  # all other types of loans with preAmm numbers of delayed payment
                k = -np.arange(freq * T + 1, 0, -1)
                im = (1 + i)**(1/freq) - 1   # interest rate compounding equivalency
                a_freqmi = (1 - (1 + i)**(- (T - preAmm)))/im  # annuity calculation
            
                R_preAmm = np.zeros((len(k), ), dtype=np.float64)
                R_preAmm[(preAmm + 1):len(k)] = S/a_freqmi  # amortizing amount paid every period

                dcf_kP = np.zeros((len(k), ), dtype=np.float64) + 1 + im
                dcf_kP = np.power(dcf_kP, k.T)
                dcf_kP[0:(preAmm + 1)] = 0
            
                C_preAmm = np.zeros((len(k), ), dtype=np.float64)
                C_preAmm = R_preAmm * dcf_kP

                # R_preAmm[1:(preAmm + 1)] = S*i
                I_preAmm = np.zeros((len(k), ), dtype=np.float64)
                I_preAmm = R_preAmm * (1 - dcf_kP)
                I_preAmm[1:(preAmm + 1)] = S*im
            
                D_preAmm = np.repeat(S, len(k)) - (np.cumsum(C_preAmm))

                dfC_preAmm = pd.DataFrame(C_preAmm)
                dfI_preAmm = pd.DataFrame(I_preAmm)
                dfD_preAmm = pd.DataFrame(D_preAmm)
                df_loan = pd.DataFrame()
                df_loan['Notional'] = dfC_preAmm
                df_loan['Interest'] = dfI_preAmm
                df_loan['Payment notional/intererest'] = dfC_preAmm + dfI_preAmm
                df_loan['Debt amount'] = dfD_preAmm
                return df_loan

## Case studies

### Bullet loan annual payments

In [23]:
# applying 4 digits floating format
format_mapping = {"Notional": "{:,.4f}", "Interest": "{:,.4f}", "Payment notional/intererest": "{:.4f}","Debt amount":"{:,.4f}"}

In [24]:
bulletLoan = loan_amortizationVectorizedV2(100, 0.05, 10, 1, 9)

In [25]:
bulletLoan.style.format(format_mapping)

,Notional,Interest,Payment notional/intererest,Debt amount
0,0.0000,0.0000,0.0000,100.0000
1,0.0000,5.0000,5.0000,100.0000
2,0.0000,5.0000,5.0000,100.0000
3,0.0000,5.0000,5.0000,100.0000
4,0.0000,5.0000,5.0000,100.0000
5,0.0000,5.0000,5.0000,100.0000
6,0.0000,5.0000,5.0000,100.0000
7,0.0000,5.0000,5.0000,100.0000
8,0.0000,5.0000,5.0000,100.0000
9,0.0000,5.0000,5.0000,100.0000


### Plain french amortizing loan annual payments

In [26]:
frenchAmortLoan = loan_amortizationVectorizedV2(100, 0.05, 10, 1, 0)

In [27]:
frenchAmortLoan.style.format(format_mapping)

,Notional,Interest,Payment notional/intererest,Debt amount
0,0.0000,0.0000,0.0000,100.0000
1,7.9505,5.0000,12.9505,92.0495
2,8.3480,4.6025,12.9505,83.7016
3,8.7654,4.1851,12.9505,74.9362
4,9.2036,3.7468,12.9505,65.7325
5,9.6638,3.2866,12.9505,56.0687
6,10.1470,2.8034,12.9505,45.9217
7,10.6544,2.2961,12.9505,35.2673
8,11.1871,1.7634,12.9505,24.0802
9,11.7464,1.2040,12.9505,12.3338


### Amortizing loan with 3 deferred annual payments period from $ t = 0$

In [28]:
frenchAmortLoan2 = loan_amortizationVectorizedV2(100, 0.05, 10, 1, 3)

In [29]:
frenchAmortLoan2.style.format(format_mapping)

,Notional,Interest,Payment notional/intererest,Debt amount
0,0.0000,0.0000,0.0000,100.0000
1,0.0000,5.0000,5.0000,100.0000
2,0.0000,5.0000,5.0000,100.0000
3,0.0000,5.0000,5.0000,100.0000
4,12.2820,5.0000,17.2820,87.7180
5,12.8961,4.3859,17.2820,74.8219
6,13.5409,3.7411,17.2820,61.2811
7,14.2179,3.0641,17.2820,47.0631
8,14.9288,2.3532,17.2820,32.1343
9,15.6753,1.6067,17.2820,16.4590


### Amortizing loan with semiannual payments

In [30]:
frenchAmortLoan3 = loan_amortizationVectorizedV2(100, 0.05, 10, 6, 0)

In [31]:
frenchAmortLoan3.style.format(format_mapping)

,Notional,Interest,Payment notional/intererest,Debt amount
0,0.0000,0.0000,0.0000,100.0000
1,1.2983,0.8165,2.1148,98.7017
2,1.3089,0.8059,2.1148,97.3928
3,1.3196,0.7952,2.1148,96.0733
4,1.3303,0.7844,2.1148,94.7429
5,1.3412,0.7736,2.1148,93.4017
6,1.3522,0.7626,2.1148,92.0495
7,1.3632,0.7516,2.1148,90.6863
8,1.3743,0.7404,2.1148,89.3120
9,1.3856,0.7292,2.1148,87.9265
